In [2]:
from utils.color_generator import ColorGenerator
import hashlib
from copy import copy
from dataclasses import dataclass

import cv2
from tqdm import tqdm
import mxnet as mx
import numpy as np
from gluoncv import model_zoo, data, utils
from gluoncv.data.transforms.pose import detector_to_alpha_pose, heatmap_to_coord_alpha_pose

color_generator = ColorGenerator()

def draw_rectangle(image, x1, y1, w, h, label):
    # For bounding box
    img = copy(image)
    color = color_generator.id_to_random_color(label)
    text_color = (0, 0, 0)
    x2 = x1 + w
    y2 = y1 + h

    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    # For the text background
    # Finds space required by the text so that we can put a background with that amount of width.
    (w, h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

    # Prints the text.
    img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), color, -1)
    img = cv2.putText(img, str(label), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)

    # For printing text
    # img = cv2.putText(img, 'test', (x1, y1),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
    return img


def view_image(img):
    cv2.imshow('image', img)
    # cv2.waitKey(0)


@dataclass
class Label:
    row_id: int
    frame: int
    person_id: int
    x1: int
    y1: int
    w: int
    h: int




/home/rafa/anaconda3/envs/sportstrack/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [3]:
gpu= mx.gpu()
pose_net = model_zoo.get_model('alpha_pose_resnet101_v1b_coco', pretrained=True, ctx=gpu)

img_path = r'/home/rafa/PycharmProjects/detector-preview/data/v_i2_L4qquVg0_c009/img1/{frame:06d}.jpg'
gt_path = r'/home/rafa/PycharmProjects/detector-preview/data/v_i2_L4qquVg0_c009/gt/gt.txt'


[15:20:08] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8600 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [4]:
import pandas as pd

gt = pd.read_csv(gt_path, names=['frame', 'person_id', 'x1', 'y1', 'w', 'h', 'temp1', 'temp2', 'temp3'])
gt = gt.drop(['temp1', 'temp2', 'temp3'], axis=1)
gt[((gt['frame']==3) & (gt['person_id']==4))]


,frame,person_id,x1,y1,w,h
38,3,4,748,542,60,102


In [5]:
frame_gt = gt[gt['frame'] == 2]
frame_gt.shape

(17, 6)

In [7]:
for frame in tqdm(range(1, 489)):
    img = cv2.imread(img_path.format(frame=frame))

    labels = [Label(*rec) for rec in gt[gt['frame'] == frame].to_records()]
    bboxes = np.array([[label.x1, label.y1, label.x1 + label.w, label.y1 + label.h] for label in labels], dtype=np.float64)
    scores = mx.nd.array(np.array([np.ones(bboxes.shape[0])[:, None]]))
    classes= mx.nd.array(np.array([np.zeros(bboxes.shape[0])[:, None]]), ctx=gpu)
    det_labels = mx.nd.array(np.array([label.person_id for label in labels]))
    bboxes = mx.nd.array(np.array([bboxes[:]]), ctx=gpu)
    colors = {label.person_id: color_generator.id_to_random_color(label.person_id, normalized=True) for label in labels}
    pose_input, upscale_bbox =  detector_to_alpha_pose(img, classes, scores, bboxes, ctx=gpu)
    pose_input = pose_input.copyto(gpu)
    predicted_heatmap = pose_net(pose_input)
    upscale_bbox = mx.nd.array(upscale_bbox, ctx=gpu)
    pred_coords, confidence = heatmap_to_coord_alpha_pose(predicted_heatmap, upscale_bbox, ctx=gpu)
    ax = utils.viz.cv_plot_keypoints(img, pred_coords[4:5], confidence[4:5],
                                     classes[:,4:5], bboxes[:,4:5], scores[:,4:5],
                                     box_thresh=0.5, keypoint_thresh=0.2,)
                                     # labels=det_labels, colors=colors)
    cv2.imshow('frame', ax)
    # img = utils.viz.cv_plot_bbox(img, bboxes, scores[0], labels=det_labels, colors=colors)
    # cv2.imshow('frame', img)
    if cv2.waitKey(1) == ord('q'):
        break
    #
    #
    # view_image(img)
cv2.destroyAllWindows()
cv2.waitKey(1)


  0%|          | 0/488 [00:00<?, ?it/s]QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread (0x8d02480) is not the object's thread (0xa286db0).
Cannot move to target thread (0x8d02480)

QObject::moveToThread: Current thread

KeyboardInterrupt: 

In [ ]:
pose_input, upscale_bbox = detector_to_alpha_pose(img, classes, scores, bboxes)
print(pose_input.shape, upscale_bbox.shape)

In [ ]:
predicted_heatmap = pose_net(pose_input)
pred_coords, confidence = heatmap_to_coord_alpha_pose(predicted_heatmap, upscale_bbox)
print(pred_coords.shape, confidence.shape)

In [ ]:
ax = utils.viz.cv_plot_keypoints(img, pred_coords, confidence,
                                 classes, bboxes, scores,
                                 box_thresh=0.5, keypoint_thresh=0.2)
view_image(ax)
cv2.destroyAllWindows()